通过[TensorFlow入门教程集合](http://blog.csdn.net/jdbc/article/details/52402302) 学习的笔记

# 基本知识

## graph 和 session

（1）tensorflow是基于图的计算，graph是一个图，session是图的运行环境，下面是一个图：![](image/graph.jfif) 图需要放在session中运行才能获取结果


（2）概念介绍：
1. Tensor: 类型化的多维数组，输入输出，线
2. Operation: 执行计算的单元，图中的节点
3. Graph: 一张包含Tensor和Operation的图，
4. Session： 会话上下文，用于执行图


（3）举例：
下面定义了一个图，（tensorflow会自动定义）

In [1]:
import tensorflow as tf
graph = tf.Graph()
with graph.as_default():
    foo = tf.Variable(3, name='foo')
    bar = tf.Variable(2, name='bar')
    result = foo + bar
    initialize = tf.global_variables_initializer()
    
print(result)

Tensor("add:0", shape=(), dtype=int32)


In [6]:
with tf.Session(graph=graph) as sess:
    sess.run(initialize)
    print(sess.run(result))

5


## 数据结构

（1） rank：数据维度

Rank|Math entity|Python example
----|-----------|--------------
0|Scalar(magnitude only)|s = 438
1|Vector(magnitude and direction)| v = [1.1, 2.2, 3.3]
2|Matrix(table of numbers)|m = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
3|3-Tensor(cube of numbers)|t = [[[2],[4],[6]], [[8],[12],[14]]]
n| n-Tensor(you get the idea)|...


（2）shape：指tensor每个维度数据个数

Rank|Shape|Dimension number|Example
----|-----|----------------|-------
0|[]|0-D|A 0-D tensor.A scalar.
1|[D0]|1-D|A 1-D tensor with shape[5]
2|[D0,D1]|2-D|A 2-D tensor with shape[3, 4]
3|[D0,D1,D2]|3-D|A 3-D tensor with shape[1, 4, 3]
n|[D0,D1,...Dn]|n-D|A tensor with shape[D0,D1,...Dn]


（3）data type：数据类型

Data Type|Python type|Description
---------|-----------|-----------
DT_FLOAT|tf.float32|32位浮点数
DT_DOUBLE|tf.float64|64位浮点数
DT_INT8|tf.int8|8位有符号整数
DT_INT16|tf.int16|16位有符号整数
DT_INT32|tf.int32|32位有符号整数
DT_INT64|tf.int64|64位有符号整数
DT_UINT8|tf.uint8|8位无符号整数
DT_UINT16|tf.uint16|16位无符号整数
DT_STRING|tf.string|变长数组，每一个元素都是一个byte数组
DT_BOOL|tf.bool|boolean类型
DT_COMPLEX64|tf.complex64|complex number made of two 32 bits floating points
DT_COMPLEX128|tf.complex128|complex number made of two 64 bits floating points
DT_QINT8|tf.qint8|8 bits signed integer used in quantized Ops
DT_QINT32|tf.qint32|32 bits signed integer used in quantized Ops
DT_QUINT8|tf.quint8|8 bits unsigned integer used in quantized Ops

## 变量

In [7]:
weights = tf.Variable(tf.random_normal([784, 200], stddev=0.35),name="weights")
biases = tf.Variable(tf.zeros([200]),name='biases')

（1）介绍
`tf.random_normal`是生成一个正态分布的tensor,`shape`是第一个参数，`stddev`是标准差`tf.zeros`是生成一个全零的tensor。


（2）初始化
 初始化的时候会分配空间，初始赋值。在计算前需要初始化所有的变量，一般会在定义图的时候定义`global_variables_initializer`，在session运算是初始化所有变量。如果需要初始化部分变量，使用`variables_initializer`

In [10]:
a = tf.Variable([1], name='a')
b = tf.Variable([2], name='b')
Init_ab = tf.variables_initializer([a,b],name='init_ab')

In [11]:
init_all = tf.global_variables_initializer()

（3） 常量和变量的区别，常量会在图载入的时候跟随一起载入。变量在每个session中都是单独保存的。


（4） 命名
定义变量的时候尽量指定好每个变量的名字



## placeholders 和feed_dict

当定一个图的时候，可能有的值需要从输入数据中取得，placeholder的定义：`tf.placeholder(dtype,shape=None,name=None)`

In [12]:
x = tf.placeholder(tf.float32)

In [14]:
foo = tf.placeholder(tf.int32,shape=[1],name='foo')
bar = tf.constant(2,name='bar')
result = foo + bar
with tf.Session() as sess:
    print(sess.run(result, {foo: [1]}))

[3]


## MNIST识别实例

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

通过上面的语句或者[链接](http://yann.lecun.com/exdb/mnist/)获取数据集

使用tensorflow进行graph的构建时，大致需要以下几步

1. 为输入X和输出Y定义placeholder
2. 定义权重W
3. 定义模型结构
4. 定义损失函数
5. 定义优化算法


开始步骤：

（1） 导入需要的包


（2） 定义X和y的placeholder，以及W，b的Variable（None表示任意维度，一般是min-batch的batch size。而W定义是shape为784，10，rank为2的Variable，b是shape为10，rank为1的Variable）

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)  
x = tf.placeholder(tf.float32,[None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


（3） 定义模型

In [2]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

（4）定义损失函数,变量初始化器

In [3]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y),reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
init = tf.global_variables_initializer()

至此graph定义完毕，下面开始计算

In [7]:
sess = tf.Session()
sess.run(init)
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

迭代了1000次，每次100个样本，`mnist.train.next_batch`就是生成下一个batch的数据，接下来验证训练结果，使用单纯的正确率验证

In [9]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))
sess.close()

0.9166


# 附录1 

In [1]:
import tensorflow as tf

In [2]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(0.2)
print(node1)
print(node2)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)


In [3]:
with tf.Session() as sess:
    print(sess.run([node1, node2]))

[3.0, 0.2]


In [4]:
from __future__ import print_function
node3 = tf.add(node1, node2)
print(node3)
with tf.Session() as sess:
    print("Session run :", sess.run(node3))

Tensor("Add:0", shape=(), dtype=float32)
Session run : 3.2


In [5]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
add_node = a + b

In [6]:
with tf.Session() as sess:
    print(sess.run(add_node, {a: 3.4, b: 0.2}))
    print(sess.run(add_node, {a: [0.2, 0.3], b: [0.4, 0.5]}))

3.6
[ 0.60000002  0.80000001]


In [7]:
add_and_triple = add_node * 3.
with tf.Session() as sess:
    print(sess.run(add_and_triple, {a: 2, b: 3}))

15.0


In [8]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32) 
x = tf.placeholder(tf.float32) 
linear_model = W*x + b

初始化变量

In [9]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(linear_model, {x: [1.2, .3, -2]}))

[ 0.06       -0.21000001 -0.90000004]


In [10]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(loss, {x: [0.2, .5, 1], y: [3., 2, 0.1]}))

15.1301


In [11]:
fixW = tf.assign(W, [.1])
fixb = tf.assign(b, [-.1])
with tf.Session() as sess:
    sess.run(init)
    sess.run([fixW, fixb])
    print(sess.run(loss, {x: [0.2, .5, 1], y: [3., 2, 0.1]}))

13.6989


In [12]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
with tf.Session() as sess:
    sess.run(init)
    sess.run(train, {x: [0.2, .5, 1], y: [3., 2, 0.1]})
    print(sess.run([W, b]))
    tf.assign(W, [ 0.33646002])
    tf.assign(b, [-0.19020002])
    sess.run([W, b])
    print(sess.run(loss, {x: [0.2, .5, 1], y: [3., 2, 0.1]}))

[array([ 0.33646002], dtype=float32), array([-0.19020002], dtype=float32)]
13.8431


In [13]:
import tensorflow as tf
a = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
y1 = a * x + b
y2 = tf.placeholder(tf.float32)
loss = tf.reduce_sum(tf.square(y1 - y2))
opt = tf.train.GradientDescentOptimizer(0.01)
train = opt.minimize(loss)
init = tf.global_variables_initializer()
train_x = [0, 1, 2]
train_y = [2, 3, 4]
with tf.Session() as sess:
    sess.run(init)
    for i in range(10000):
        sess.run(train, {x: train_x, y2: train_y})
    curr_a, curr_b, curr_loss = sess.run([a, b, loss], {x: train_x, y2: train_y})
    print(curr_a, curr_b, curr_loss)

[ 1.00000298] [ 1.99999607] 2.00231e-11


In [14]:
import numpy as np
import tensorflow as tf
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)
x_train = np.array([0, 1, 2])
y_train = np.array([2, 3, 4])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
estimator.train(input_fn=input_fn, steps=1000)
train_metrics = estimator.evaluate(input_fn=train_input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\seven\\AppData\\Local\\Temp\\tmph5nfele2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000018398BD3208>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\seven\AppData\Local\Temp\tmph5nfele2\model.ckpt.
INFO:tensorflow:loss = 52.0, step = 1
INFO:tensorflow:global_step/sec: 196.577
INFO:tensorflow:loss = 0.305773, step = 101 (0.511 sec)
INFO:tensorflow:global_step/sec: 256.558
INFO:tensorflow:loss = 0.0230626, step = 201 (0.389 sec)
I

使用tf的预估来实现Estimator

In [15]:
import numpy as np
import tensorflow as tf


def model_fn(features, labels, mode):
    a = tf.get_variable("a", [1], dtype=tf.float64)
    b = tf.get_variable("b", [1], dtype=tf.float64)
    y1 = a * features["x"] + b
    loss = tf.reduce_sum(tf.square(y1 - labels))
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = tf.group(optimizer.minimize(loss),
                    tf.assign_add(global_step, 1))
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=y1,
        loss=loss,
        train_op=train)


estimator = tf.estimator.Estimator(model_fn=model_fn)
x_train = np.array([0., 1., 2.])
y_train = np.array([2., 3., 4.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
estimator.train(input_fn=input_fn, steps=1000)
train_metrics = estimator.evaluate(input_fn=train_input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\seven\\AppData\\Local\\Temp\\tmpebk923yq', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001838CFC85F8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\seven\AppData\Local\Temp\tmpebk923yq\model.ckpt.
INFO:tensorflow:loss = 152.552423817, step = 1
INFO:tensorflow:global_step/sec: 328.057
INFO:tensorflow:loss = 0.0381845501129, step = 101 (0.308 sec)
INFO:tensorflow:global_step/sec: 339.178
INFO:tensorflow:loss = 0.00021294909699, s